<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Анализ-данных" data-toc-modified-id="Анализ-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Анализ данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Подгтовка-и-работа-с-выборками" data-toc-modified-id="Подгтовка-и-работа-с-выборками-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подгтовка и работа с выборками</a></span></li><li><span><a href="#Обучение-и-тестирование-моделей" data-toc-modified-id="Обучение-и-тестирование-моделей-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение и тестирование моделей</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [73]:
import numpy as np
import pandas as pd

import re

import nltk

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import stopwords as сс

nltk.download('wordnet','stopwords','punkt','averaged_perceptron_tagger')
from nltk.corpus import wordnet

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle

from catboost import CatBoostClassifier

import matplotlib.pyplot as plt

import time

import warnings
warnings.filterwarnings('ignore')


<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [74]:
data = pd.read_csv('/datasets/toxic_comments.csv')

### Анализ данных

In [75]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [77]:
# дубликаты
data.duplicated().sum()



0

In [78]:
#пропуски
data.isna().sum()



Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [79]:
#соотношение в целевом признаке
data.toxic.value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [80]:
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


In [81]:
#удаляю столбец Unnamed
data = data.drop(['Unnamed: 0'], axis=1)

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены корреткно. Радует, что баланс классов был изучен.
</div>

### Подготовка данных

In [82]:
# нижний регистр
data['text'] = data['text'].str.lower()


In [83]:
# буквы в нижнем и верхнем регистре + цифры
data_new = []
pattern = r'[^a-zA-Z0-9]' #r'[^a-zA-z]' [^a-zA-Z0-9]
for sentence in data.text:
  cleared_text = re.sub(pattern, " ", sentence)
  data_new.append(" ". join(cleared_text.split()))

In [84]:
data["clear_text"]=data_new
data.head()

,text,toxic,clear_text
0,explanation\nwhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,d'aww! he matches this background colour i'm s...,0,d aww he matches this background colour i m se...
2,"hey man, i'm really not trying to edit war. it...",0,hey man i m really not trying to edit war it s...
3,"""\nmore\ni can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"you, sir, are my hero. any chance you remember...",0,you sir are my hero any chance you remember wh...


In [85]:
# сэмплирую
sample_size = 90000
corpus = data.sample(n=sample_size,random_state=0).reset_index(drop=True)

In [86]:
#токенизациия и лемматизация массива текстов
lemmatizer = WordNetLemmatizer()
def lemmatizered(corpus):
  
  corpus_new = []
  for sentence in corpus:
    word_list = nltk.word_tokenize(sentence)
    corpus_new.append(' '.join([lemmatizer.lemmatize(w) for w in word_list]))
  return corpus_new

In [87]:
nltk.download('stopwords')
nltk_stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [88]:
#  словарь, где возвращается значение часть речи 
def get_wordnet_pos(word):
    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
 
    return tag_dict.get(tag, wordnet.NOUN)

In [89]:
#токенизациия и лемматизация массива текстов c учетом pos_tag и удаление стоп-слов
def get_word_text(corpus):
  
  corpus_new = []
  for sentence in corpus:
    corpus_new.append(' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence) if not w in nltk_stopwords.words('english')]))
  return corpus_new

In [90]:
#лемматизация корпуса c учетом pos_tag


In [91]:
%%time
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
 
corpus['clear_text_2'] = get_word_text(corpus['clear_text']) 


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


CPU times: user 16min 7s, sys: 1min 59s, total: 18min 7s
Wall time: 18min 9s


In [92]:
corpus.head(10)

,text,toxic,clear_text,clear_text_2
0,"""\nwell, since i am blocked, i shall temporari...",0,well since i am blocked i shall temporarily re...,well since block shall temporarily respond whe...
1,"""\n\nhahahahaha. typical. \n\nthe article is a...",0,hahahahaha typical the article is about anti p...,hahahahaha typical article anti pedophile acti...
2,which you made after i was encouraged by a med...,0,which you made after i was encouraged by a med...,make encourage mediator admin make one
3,i regard you a racist. i will request you be b...,1,i regard you a racist i will request you be ba...,regard racist request ban life
4,"""\n\na broken chair is not a chair\na broken c...",0,a broken chair is not a chair a broken chord i...,broken chair chair broken chord chord invert p...
5,now that you let screwattack how about you fuc...,1,now that you let screwattack how about you fuc...,let screwattack fuck pedant wikipedia asshole ...
6,national geographic \n\nwhy is there no mentio...,0,national geographic why is there no mention in...,national geographic mention article murdoch re...
7,pgagnon999 has used a sock puppet to remove ci...,1,pgagnon999 has used a sock puppet to remove ci...,pgagnon999 use sock puppet remove cite materia...
8,"""\n\n dhudhi family‎(‎ڈهڈی‎) \n\nthe dhudhi‎(‎...",0,dhudhi family the dhudhi are a tribe of panwar...,dhudhi family dhudhi tribe panwar parmar rajpu...
9,"""\nre-wording\naccording to the european commi...",0,re wording according to the european commissio...,word accord european commission change accord ...


### Вывод

В датасете 159 292 объекта.

Пропусков нет, явных дубликатов нет

Тексты комментариев на английском

В целевом признаке 90% объектов отрицательного класса, то есть в дальнейшем нужно будет учесть это

Избавилась от столбца Unnamed, так как он фактически дублирует индексы

Для дальнейшего использования 159571 очень большой датасет, поэтому сделала sample

Очистила тексты комментариев от ненужных знаков, леммализировала, убрала стоп-слова

Сбалансировала данные в целевом признаке

<div class="alert alert-block alert-success">
<b>Успех:</b> Очистка и лемматизация были сделаны корректно. Но лемматизацию правильно проводить только с учетом POS-tag.
</div>

## Обучение

### Подгтовка и работа с выборками

In [95]:

#разделяю выборки 
features = corpus.drop(['toxic'], axis=1) 
target = corpus.toxic

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.2, 
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=.5,
                                                                            random_state=12345)

In [96]:
#смотрю размеры выборок:
for i in [features_train, target_train, features_valid, target_valid, features_test, target_test]:
    print(i.shape)

(72000, 3)
(72000,)
(9000, 3)
(9000,)
(9000, 3)
(9000,)


In [97]:
#уменьшаю кол-во 0 в выборках train

corpus_train = corpus.iloc[target_train.index]
target_train_0 = corpus_train[corpus_train['toxic'] == 0]['toxic']
target_train_1 = corpus_train[corpus_train['toxic'] == 1]['toxic']


target_train_0_resample = target_train_0.sample(target_train_1.shape[0], random_state=12345)
target_train_resample = pd.concat([target_train_0_resample, target_train_1])

features_train_resample = corpus.iloc[target_train_resample.index]

features_train_resample, target_train_resample = shuffle(features_train_resample,
                                                         target_train_resample,
                                                         random_state=12345)

features_train_resample = features_train_resample.text 

print('Соотношение 1 и 0 в тренировочной выборке')
print(target_train_resample.value_counts(normalize=True))
print()
print(features_train_resample.shape)
print(target_train_resample.shape)

Соотношение 1 и 0 в тренировочной выборке
0    0.5
1    0.5
Name: toxic, dtype: float64

(14734,)
(14734,)


### Обучение и тестирование моделей

####  Логистическая регрессия


In [98]:
features_train = features_train.text


In [99]:
%%time

#обучение 
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200])} 
gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

print('F1 логистической регрессии ', round(lr_train_f1,2))
print('лучшие параметры', gscv.best_params_)
print()


#тестирование
#predictions_test = gscv.predict(features_test.text)
#lr_test_f1_1 = f1_score(target_test, predictions_test)
#print('F1 логистической регрессии на тесте ', round(lr_test_f1_1,2))


F1 логистической регрессии  0.76
лучшие параметры {'lr__C': 10, 'lr__max_iter': 200, 'lr__random_state': 12345, 'lr__solver': 'lbfgs'}

CPU times: user 8min 33s, sys: 6min 47s, total: 15min 20s
Wall time: 15min 21s


<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что освоила и применила пайплайн!
</div>

#### DecisionTreeClassifier

In [100]:
%%time

#обучение
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("dtc", DecisionTreeClassifier())])
    
parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__random_state': ([12345]),
              'dtc__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
dtc_train_f1 = max(mts)

print('F1 дерева решений ', round(dtc_train_f1,2))
print('лучшие параметры', gscv.best_params_)
print()

#тестирование
#predictions_test = gscv.predict(features_test.text)
#dtc_test_f1 = f1_score(target_test, predictions_test)
#print('F1 дерева решений на тесте', round(dtc_test_f1,2))

F1 дерева решений  0.62
лучшие параметры {'dtc__class_weight': 'balanced', 'dtc__max_depth': 24, 'dtc__random_state': 12345}

CPU times: user 8min 7s, sys: 2.81 s, total: 8min 10s
Wall time: 8min 10s


#### CatBoostClassifier


In [101]:
%%time

#обучение
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("cbc", CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200]),
              'cbc__class_weights':([(1, 1), (1, 11)])} 

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
cbc_train_f1 = max(mts)

print('F1 CatBoostClassifier ', round(cbc_train_f1,2))
print('лучшие параметры', gscv.best_params_)
print()



#тестирование
#predictions_test = gscv.predict(features_test.text)
#cbc_test_f1 = f1_score(target_test, predictions_test)
#print('F1 CatBoostClassifier на тесте', round(cbc_test_f1,2))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

F1 CatBoostClassifier  0.72
лучшие параметры {'cbc__class_weights': (1, 11), 'cbc__iterations': 200, 'cbc__verbose': False}

CPU times: user 29min 22s, sys: 15.9 s, total: 29min 37s
Wall time: 29min 42s


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> На тестовой выборке нужно измерить только одну – лучшую модель. Сравнение моделей нужно провести только на кросс-валидации (с одним и тем же параметром "cv") или только на валидационной выборке.
</div>

In [102]:
#создаю сводную таблицу по показателям F1
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'CatBoostClassifier'
        ]
        

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    cbc_train_f1]}
        
        #'F1 на тестовой выборке': [lr_test_f1_1,
         #                               dtc_test_f1,
         #                               cbc_test_f1]

f1_data = pd.DataFrame(data=data, index=index)

f1_data.sort_values(by='F1 на обучающей выборке', ascending=False)

,F1 на обучающей выборке
LogisticRegression,0.757958
CatBoostClassifier,0.724022
DecisionTreeClassifier,0.624130


In [106]:
#обучение
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200]),
              'lr__class_weight': (['balanced'])} 
gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

#тестирование
predictions_test = gscv.predict(features_test.text)
lr_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 логистической регрессии =', round(lr_test_f1,2))


финальный F1 логистической регрессии = 0.77


#### Вывод

Разделила датасет на выборки.

Обучила 3 модели с разными гиперпараметрами.

Наилучшей моделью стала LogisticRegression, которая на тестовой выборке  показала F1 = 0.77.

## Вывод

Я построила модель со значением метрики качества F1 = 0,77. Таким образом, удалось достичь нужного показателя.Интернет-магазину «Викишоп» рекомендуется использовать модель LogisticRegression, которая будет искать токсичные комментарии и отправлять их на модерацию. 

:)